# Sweeps notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint
import os
import re
import json
import sys
import wandb
from model import *
from pytorch_lightning.loggers import WandbLogger

c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
# Define sweep config
sweep_config = {
    'method': 'random' # random, grid, or bayes
    }

In [3]:
# Define metric
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   # minimize or maximize
    }
# add in sweep_config
sweep_config['metric'] = metric

In [4]:
epochs = 100
architecture = 'CNN'
dataset = 'Poems'
# Define hyperparameter space
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'embedding_dim': {
        'values': [256, 512]
        },
    }
# we can indicate the distribution for continuous variables
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.1
      }
    })
# we set values that we want to track but don't want to change, just indicate 1 value
parameters_dict.update({
    'epochs': {'value': epochs},
    "architecture":{'value': architecture},
    "dataset": {'value': dataset},        
    })
# add params in sweep_config
sweep_config['parameters'] = parameters_dict

In [5]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'architecture': {'value': 'CNN'},
                'dataset': {'value': 'Poems'},
                'embedding_dim': {'values': [256, 512]},
                'epochs': {'value': 100},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [6]:
# Log in to weights and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: jjdsantiago3 (msds_mlops2023_lt2). Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_id = wandb.sweep(sweep_config, project="mlo-final-project")

Create sweep with ID: 77qlslpj
Sweep URL: https://wandb.ai/msds_mlops2023_lt2/mlo-final-project/sweeps/77qlslpj


In [8]:
# define hyperparam search function
def hyperparam_search(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # use config params
        learning_rate = config.learning_rate
        optimizer = config.optimizer
        embedding_dim = config.embedding_dim
        epochs = config.epochs

        # define model
        _model = poem_classifier_model()

        _model.load_data()
        _model.preprocess()
        _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)

        for i in range(epochs):
            wandb.log({"val_loss": _model.trained_model.history['val_loss'][i], 
                       "val_acc": _model.trained_model.history['val_acc'][i]})

wandb.agent(sweep_id, hyperparam_search, count=5) # count - num iters

wandb: Agent Starting Run: ntyrjk87 with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 512
wandb: 	epochs: 100
wandb: 	learning_rate: 0.043743982189250255
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Epoch 1/100
21/21 [==============================] - 1s 26ms/step - loss: 1.4227 - acc: 0.3169 - val_loss: 1.3486 - val_acc: 0.3452
Epoch 2/100
21/21 [==============================] - 0s 18ms/step - loss: 1.1188 - acc: 0.5007 - val_loss: 1.6841 - val_acc: 0.3036
Epoch 3/100
21/21 [==============================] - 0s 18ms/step - loss: 0.9486 - acc: 0.6338 - val_loss: 1.9474 - val_acc: 0.3750
Epoch 4/100
21/21 [==============================] - 0s 18ms/step - loss: 0.8841 - acc: 0.6697 - val_loss: 1.8002 - val_acc: 0.3929
Epoch 5/100
21/21 [==============================] - 0s 18ms/step - loss: 0.7686 - acc: 0.7160 - val_loss: 2.1155 - val_acc: 0.3631
Epoch 6/100
21/21 [==============================] - 0s 19ms/step - loss: 0.5889 - acc: 0.7683 - val_loss: 2.1450 - val_acc: 0.3929
Epoch 7/100
21/21 [==============================] - 0s 18ms/step - loss: 0.4999 - acc: 0.8191 - val_loss: 2.6113 - val_acc: 0.3810
Epoch 8/100
21/21 [==============================] - 0s 20ms/step - loss: 0.

val_acc,▂▅▆▅▅▇▃▅▇▃▃▅▂▆▅▇▃▅▅▇█▅▅▅▁▃▇▂▃▃▄▄▆▄▃▆▂▃▄▆
val_loss,▁▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██
val_acc,0.3869
val_loss,9.25744


wandb: Agent Starting Run: r1vxtd20 with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 256
wandb: 	epochs: 100
wandb: 	learning_rate: 0.021222360230203272
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: hd4e465l with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 256
wandb: 	epochs: 100
wandb: 	learning_rate: 0.09055556227579983
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

21/21 [==============================] - 1s 18ms/step - loss: 1.5657 - acc: 0.3019 - val_loss: 1.4709 - val_acc: 0.3929
Epoch 2/100
21/21 [==============================] - 0s 12ms/step - loss: 1.1201 - acc: 0.5366 - val_loss: 1.5328 - val_acc: 0.3750
Epoch 3/100
21/21 [==============================] - 0s 12ms/step - loss: 0.8668 - acc: 0.6831 - val_loss: 1.7696 - val_acc: 0.4107
Epoch 4/100
21/21 [==============================] - 0s 12ms/step - loss: 0.9867 - acc: 0.6801 - val_loss: 2.1333 - val_acc: 0.3929
Epoch 5/100
21/21 [==============================] - 0s 11ms/step - loss: 0.9091 - acc: 0.6921 - val_loss: 2.5733 - val_acc: 0.4107
Epoch 6/100
21/21 [==============================] - 0s 11ms/step - loss: 0.9019 - acc: 0.7205 - val_loss: 2.8820 - val_acc: 0.3274
Epoch 7/100
21/21 [==============================] - 0s 12ms/step - loss: 0.8254 - acc: 0.7250 - val_loss: 3.1038 - val_acc: 0.3036
Epoch 8/100
21/21 [==============================] - 0s 11ms/step - loss: 0.5971 - acc: 

val_acc,▇█▄▅▂▄▁▄▄▃▅▆▄▄▃▃▂▃▂▁▂▃▅▄█▄▅▃▄▃▄▄▄▄▃▂▅▄▄▄
val_loss,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇██▇██▇▇███
val_acc,0.33929
val_loss,16.60823
